# Machine Translation

### Data Processing and Analysis

In [ ]:
# Requirements

# pip install tensorflow-metal
# pip install torch-sparse==0.6.12
# pip install tensorflow-macos
# pip install sentencepiece
# pip install bpemb
# pip install -U torchtext
# pip install --upgrade pyarrow
# pip install datasets
# pip install onnxruntime
# pip install evaluate
# pip install accelerate -U


In [ ]:
#### Run this cell

!pip install evaluate
!pip install accelerate -U
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 54.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Foun

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [ ]:
import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp
device = xm.xla_device()

print(device)

xla:0


In [1]:
#### Run this cell

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
import sentencepiece as spm
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from tqdm import tqdm
from tqdm import trange

# For model

import torch
import torch.nn as nn
import torch.optim as optim
import transformers
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
# from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
# from datasets import load_metric
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from transformers import MBartTokenizer, MBartForConditionalGeneration
from tqdm.auto import tqdm
# from bpemb import BPEmb
import evaluate
from torch.nn.utils.rnn import pad_sequence
import torch.nn.functional as F

In [ ]:
# Selecting the first 9000 rows of the data

num_rows = 15000

# All language dataset

# data_bulg = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_bulgarian.csv',nrows=num_rows)
# data_czec = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_czech.csv',nrows=num_rows)
# data_dutch = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Machine Translation/english_dutch.csv',nrows=num_rows)
# data_est = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_estonian.csv',nrows=num_rows)
# data_fin = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Machine Translation/english_finnish.csv',nrows=num_rows)
# data_fre = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_french.csv',nrows=num_rows)
# data_ger = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Machine Translation/english_german.csv',nrows=num_rows)
# data_greek = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Machine Translation/english_greek.csv',nrows=num_rows)
# data_hung = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_hungarian.csv',nrows=num_rows)
# data_itl = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Machine Translation/english_italian.csv',nrows=num_rows)
# data_lat = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_latvia.csv',nrows=num_rows)
# data_lit = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_lithunian.csv',nrows=num_rows)
# data_pol = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Machine Translation/english_polish.csv',nrows=num_rows)
# data_port = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_portuguese.csv',nrows=num_rows)
# data_rom = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_romanian.csv',nrows=num_rows)
# data_slo  = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_slovak.csv',nrows=num_rows)
# data_slove = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_slovene.csv',nrows=num_rows)
# data_spa = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Machine Translation/english_spanish.csv',nrows=num_rows)
# data_swe = pd.read_csv('/Users/marianivethaantonypushparaj/Downloads/Machine Translation/english_swedish.csv',nrows=num_rows)

# Selected Languages (3)

data_dutch = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Machine Translation/english_dutch.csv',nrows=num_rows)
data_ger = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Machine Translation/english_german.csv',nrows=num_rows)
data_spa = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Machine Translation/english_spanish.csv',nrows=num_rows)

In [ ]:
# gpus = tf.config.list_physical_devices("GPU")
# if gpus:
#     for gpu in gpus:
#         print(gpu)
# else:
#     print('GPU not found')

In [ ]:
# Remove duplicates, drop NaN values and reset index

data_dutch = data_dutch.drop_duplicates(inplace=False).reset_index(drop=True).dropna().reset_index(drop=True)
data_ger = data_ger.drop_duplicates(inplace=False).reset_index(drop=True).dropna().reset_index(drop=True)
data_spa = data_spa.drop_duplicates(inplace=False).reset_index(drop=True).dropna().reset_index(drop=True)


In [ ]:
print("Dutch dataset\n",data_dutch.dtypes)
print("German dataset\n",data_ger.dtypes)
print("Spannish dataset\n",data_spa.dtypes)

Dutch dataset
 English    object
Dutch      object
dtype: object
German dataset
 English    object
German     object
dtype: object
Spannish dataset
 English    object
Spanish    object
dtype: object


In [ ]:
# Check the data type of entries in data

# Dutch
print("Type of Dutch data:",type(data_dutch['Dutch'][0]))
print("Type of English data:",type(data_dutch['English'][0]))

# German
print("Type of German data:",type(data_ger['German'][0]))
print("Type of English data:",type(data_ger['English'][0]))

# Spanish
print("Type of Spanish data:",type(data_spa['Spanish'][0]))
print("Type of English data:",type(data_spa['English'][0]))

Type of Dutch data: <class 'str'>
Type of English data: <class 'str'>
Type of German data: <class 'str'>
Type of English data: <class 'str'>
Type of Spanish data: <class 'str'>
Type of English data: <class 'str'>


In [ ]:
# Convert the datatype of all entries to string

data_dutch = data_dutch.convert_dtypes(convert_string=True)
data_ger = data_ger.convert_dtypes(convert_string=True)
data_spa = data_spa.convert_dtypes(convert_string=True)

In [ ]:
print("Dutch dataset\n",data_dutch.dtypes)

print("German dataset\n",data_ger.dtypes)

print("Spannish dataset\n",data_spa.dtypes)

Dutch dataset
 English    string[python]
Dutch      string[python]
dtype: object
German dataset
 English    string[python]
German     string[python]
dtype: object
Spannish dataset
 English    string[python]
Spanish    string[python]
dtype: object


In [ ]:
# Shuffle the data

data_dutch = data_dutch.sample(frac=1).reset_index(drop=True)
data_ger = data_ger.sample(frac=1).reset_index(drop=True)
data_spa = data_spa.sample(frac=1).reset_index(drop=True)

In [ ]:
# Adding tokens to each sentence

# Dutch
tag = "<nl_XX>"
data_dutch['Dutch'] = [f"{tag} {sentence}" for sentence in data_dutch['Dutch']]

# German
tag = "<de_DE>"
data_ger['German'] = [f"{tag} {sentence}" for sentence in data_ger['German']]

# Spanish
tag = "<es_XX>"
data_spa['Spanish'] = [f"{tag} {sentence}" for sentence in data_spa['Spanish']]


In [ ]:
# Splitting the data into train, test and validation

# Dutch
train_dutch, temp_dutch = train_test_split(data_dutch, test_size=0.2, random_state=42)
valid_dutch, test_dutch = train_test_split(temp_dutch, test_size=0.5, random_state=42)

# German
train_german, temp_german = train_test_split(data_ger, test_size=0.2, random_state=42)
valid_german, test_german = train_test_split(temp_german, test_size=0.5, random_state=42)

# Spanish
train_spanish, temp_spanish = train_test_split(data_spa, test_size=0.2, random_state=42)
valid_spanish, test_spanish = train_test_split(temp_spanish, test_size=0.5, random_state=42)

In [ ]:
train_dutch = train_dutch.reset_index(drop=True)
valid_dutch = valid_dutch.reset_index(drop=True)
test_dutch = test_dutch.reset_index(drop=True)

train_german = train_german.reset_index(drop=True)
valid_german = valid_german.reset_index(drop=True)
test_german = test_german.reset_index(drop=True)

train_spanish = train_spanish.reset_index(drop=True)
valid_spanish = valid_spanish.reset_index(drop=True)
test_spanish = test_spanish.reset_index(drop=True)

In [ ]:
train_dutch.to_csv('Dutch_train.csv')
valid_dutch.to_csv('Dutch_validation.csv')
test_dutch.to_csv('Dutch_test.csv')

train_german.to_csv('German_train.csv')
valid_german.to_csv('German_validation.csv')
test_german.to_csv('German_test.csv')


train_spanish.to_csv('Spanish_train.csv')
valid_spanish.to_csv('Spanish_validation.csv')
test_spanish.to_csv('Spanish_test.csv')

In [ ]:
# Generate a combination of training, validation and test data

train_data = pd.DataFrame()
train_data['source'] = pd.concat([train_dutch['English'],train_german['English'],train_spanish['English']])
train_data['target'] = pd.concat([train_dutch['Dutch'],train_german['German'],train_spanish['Spanish']])

val_data = pd.DataFrame()
val_data['source'] = pd.concat([valid_dutch['English'],valid_german['English'],valid_spanish['English']])
val_data['target'] = pd.concat([valid_dutch['Dutch'],valid_german['German'],valid_spanish['Spanish']])

test_data = pd.DataFrame()
test_data['source'] = pd.concat([test_dutch['English'],test_german['English'],test_spanish['English']])
test_data['target'] = pd.concat([test_dutch['Dutch'],test_german['German'],test_spanish['Spanish']])

In [ ]:
train_data.to_csv('Train_data.csv')
val_data.to_csv('Validation_data.csv')
test_data.to_csv('Test_data.csv')

In [ ]:
# Reset index

train_data = train_data.sample(frac=1).reset_index(drop=True)
val_data = val_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

### Testing Pre-trained models from BERT

In [ ]:
# Mbart initialization

model_mbart = "facebook/mbart-large-50-one-to-many-mmt"
tokenizer = AutoTokenizer.from_pretrained(model_mbart, src_lang="en_XX")
mbart_basic = AutoModelForSeq2SeqLM.from_pretrained(model_mbart)
mbart_basic.to_bettertransformer()


The BetterTransformer implementation does not support padding during training, as the fused kernels do not support attention masks. Beware that passing padded batched data during training may result in unexpected outputs. Please refer to https://huggingface.co/docs/optimum/bettertransformer/overview for more details.


MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250054, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayerBetterTransformer(
          (act_fn_callable): ReLU()
        )
      )
      (layernorm_embedding): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): MBartDecoder(
      (embed_tokens): Embedding(250054, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartDecoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          

In [ ]:
# Example usage of mbart

# tokenizer = AutoTokenizer.from_pretrained(model_mbart, src_lang="en_XX")
# mbart_basic = AutoModelForSeq2SeqLM.from_pretrained(model_mbart)
# encoded_en = tokenizer(en_text, return_tensors="pt")
# generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.lang_code_to_id["de_FI"])
# tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

In [ ]:
# Create a separate dataframe to store the results for comparison

test_outputs_df = test_data.copy()
test_outputs_df['MBart-basic_pre-trained version'] = None
test_outputs_df

,source,target,MBart-basic_pre-trained version
0,But we feel that it would be wiser not to set ...,"<es_XX> Sin embargo, a nuestro juicio, es prud...",None
1,Subject: 'Mainstreaming' in EU aid policy In 1...,<de_DE> Betrifft: Einbeziehung der Gleichstell...,None
2,I would like to emphasise the strong political...,<de_DE> Ich möchte besonders auf die vom Europ...,None
3,It would be very difficult for the Community t...,<de_DE> Es wäre angesichts der unterschiedlich...,None
4,We are delighted today to see the progress thi...,<nl_XX> Het doet ons genoegen dat deze gedacht...,None
...,...,...,...
4417,"In the stance that we took at Seattle, why suc...",<nl_XX> Waarom werd er in het door ons in Seat...,None
4418,The vote will take place tomorrow at 11.30 a.m.,<nl_XX> De stemming vindt morgen om 11.30 uur ...,None
4419,On Friday of this week we have one single item...,<es_XX> En el viernes de esta semana tenemos u...,None
4420,The European Union voted against that motion.,<de_DE> Die Europäische Union stimmte gegen di...,None


In [ ]:
model_mbart = "facebook/mbart-large-50-one-to-many-mmt"
# tokenizer = AutoTokenizer.from_pretrained(model_mbart, src_lang="en_XX")
mbart_basic = AutoModelForSeq2SeqLM.from_pretrained(model_mbart)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [ ]:
for i in tqdm(range(len(test_outputs_df)), desc="Translating Test data"):
    encoded_en_text = tokenizer(test_data['source'].iloc[i], return_tensors="pt",truncation = True)
    generated_tokens = mbart_basic.generate(**encoded_en_text, forced_bos_token_id=tokenizer.lang_code_to_id[test_data['target'].iloc[i][0:7][1:-1]])
    test_outputs_df['MBart-basic_pre-trained version'].iloc[i] = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

Translating Test data:   0%|          | 0/4422 [00:00<?, ?it/s]/var/folders/28/gj9cb4w95rb4sbjdp44jc_hh0000gn/T/ipykernel_37528/743272147.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_outputs_df['MBart-basic_pre-trained version'].iloc[i] = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
Translating Test data: 100%|██████████| 4422/4422 [13:51:36<00:00, 11.28s/it]  


In [ ]:
test_outputs_df.to_csv('Test output_evaluation.csv')

In [ ]:
bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

predictions = test_outputs_df['MBart-basic_pre-trained version'].tolist()
references = test_outputs_df['target'].apply(lambda x: x[7:]).tolist()


bleu_score = bleu_metric.compute(predictions=predictions_bleu, references=references_bleu)
print(f"BLEU score: {bleu_score}")

# Compute ROUGE score
rouge_score = rouge_metric.compute(predictions=predictions, references=references)
print(f"ROUGE score: {rouge_score}")

# Compute METEOR score
meteor_score = meteor_metric.compute(predictions=predictions, references=references)
print(f"METEOR score: {meteor_score}")


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/marianivethaantonypushparaj/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/marianivethaantonypushparaj/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/marianivethaantonypushparaj/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU score: {'bleu': 0.21091772010050708, 'precisions': [0.4895216421320479, 0.25970948511507713, 0.15744713975732455, 0.1007390990514726], 'brevity_penalty': 0.9953250437306422, 'length_ratio': 0.9953359373606577, 'translation_length': 111611, 'reference_length': 112134}
ROUGE score: {'rouge1': AggregateScore(low=Score(precision=0.5325741109296083, recall=0.5277966446996597, fmeasure=0.5251945053148783), mid=Score(precision=0.5379798862499148, recall=0.5333662313065007, fmeasure=0.530634450986424), high=Score(precision=0.5435129597779425, recall=0.5388004088050843, fmeasure=0.535734241956761)), 'rouge2': AggregateScore(low=Score(precision=0.298886911916712, recall=0.2959950282698386, fmeasure=0.29490486027413665), mid=Score(precision=0.3049592954010911, recall=0.3019640005615082, fmeasure=0.30078566708034277), high=Score(precision=0.3112179150288727, recall=0.30800270136993846, fmeasure=0.30695373092298567)), 'rougeL': AggregateScore(low=Score(precision=0.4807350555992607, recall=0.47

# Fine Tuning MBart

In [ ]:
##### Run this cell

# Adding English Token

# train_data = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Train_data.csv')
# val_data = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Validation_data.csv')
# test_data = pd.read_csv('/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/Test_data.csv')

train_data = pd.read_csv('Train_data.csv')
val_data = pd.read_csv('Validation_data.csv')
test_data = pd.read_csv('Test_data.csv')

train_data = train_data.sample(frac=1).reset_index(drop=True).drop(columns=['Unnamed: 0'])
val_data = val_data.sample(frac=1).reset_index(drop=True).drop(columns=['Unnamed: 0'])
test_data = test_data.sample(frac=1).reset_index(drop=True).drop(columns=['Unnamed: 0'])

# tag = "<en_XX>"
# train_data['source'] = [f"{tag} {sentence}" for sentence in train_data['source']]
# val_data['source'] = [f"{tag} {sentence}" for sentence in val_data['source']]
# test_data['source'] = [f"{tag} {sentence}" for sentence in test_data['source']]

In [2]:
##### Run this cell

model = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50-one-to-many-mmt')
tokenizer = MBartTokenizer.from_pretrained('facebook/mbart-large-50-one-to-many-mmt')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/528 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


In [4]:
##### Run this cell

def preprocess_texts(texts, src_lang_code, tgt_lang_code):
    processed_src_texts = []
    processed_tgt_texts = []
    for src_text, tgt_text in texts:
        src_text = f"{src_text} {tokenizer.eos_token} {src_lang_code}"
        tgt_text = f"{tgt_lang_code} {tgt_text} {tokenizer.eos_token}"
        processed_src_texts.append(src_text)
        processed_tgt_texts.append(tgt_text)
    return processed_src_texts, processed_tgt_texts


In [5]:
##### Run this cell


class TranslationDataset(Dataset):
    def __init__(self, texts, tokenizer, src_lang_code, tgt_lang_code):
        self.texts = texts
        self.tokenizer = tokenizer
        self.src_lang_code = src_lang_code
        self.tgt_lang_code = tgt_lang_code
        self.src_texts, self.tgt_texts = preprocess_texts(texts, src_lang_code, tgt_lang_code)

    def __len__(self):
        return len(self.src_texts)

    def __getitem__(self, idx):
        src_text = self.src_texts[idx]
        tgt_text = self.tgt_texts[idx]

        inputs = self.tokenizer(src_text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
        labels = self.tokenizer(tgt_text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)

        input_ids = inputs.input_ids.squeeze()
        attention_mask = inputs.attention_mask.squeeze()
        labels = labels.input_ids.squeeze()

        labels[labels == self.tokenizer.pad_token_id] = -100

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": labels
        }


In [6]:
##### Run this cell

train_dutch = pd.read_csv('Dutch_train.csv')
valid_dutch = pd.read_csv('Dutch_validation.csv')

train_german = pd.read_csv('German_train.csv')
valid_german = pd.read_csv('German_validation.csv')

train_spanish = pd.read_csv('Spanish_train.csv')
valid_spanish = pd.read_csv('Spanish_validation.csv')

In [7]:
##### Run this cell

train_dutch = train_dutch[0:3000]
train_german = train_german[0:3000]
train_spanish = train_spanish[0:3000]

valid_dutch = valid_dutch[0:500]
valid_german = valid_german[0:500]
valid_spanish = valid_spanish[0:500]

In [8]:
##### Run this cell

english_dutch_texts = list(zip(train_dutch['English'].tolist(), train_dutch['Dutch'].tolist()))  # For Dutch translation
english_spanish_texts = list(zip(train_spanish['English'].tolist(), train_spanish['Spanish'].tolist()))
english_german_texts = list(zip(train_german['English'].tolist(), train_german['German'].tolist()))

datasets = [
    TranslationDataset(english_dutch_texts, tokenizer, "en_XX", "nl_XX"),
    TranslationDataset(english_spanish_texts, tokenizer, "en_XX", "es_XX"),
    TranslationDataset(english_german_texts, tokenizer, "en_XX", "de_DE")
]

train_combined_dataset = ConcatDataset(datasets)
train_dataloader = DataLoader(train_combined_dataset, batch_size=4, shuffle=True)

english_dutch_texts = list(zip(valid_dutch['English'].tolist(), valid_dutch['Dutch'].tolist()))  # For Dutch translation
english_spanish_texts = list(zip(valid_spanish['English'].tolist(), valid_spanish['Spanish'].tolist()))
english_german_texts = list(zip(valid_german['English'].tolist(), valid_german['German'].tolist()))

datasets = [
    TranslationDataset(english_dutch_texts, tokenizer, "en_XX", "nl_XX"),
    TranslationDataset(english_spanish_texts, tokenizer, "en_XX", "es_XX"),
    TranslationDataset(english_german_texts, tokenizer, "en_XX", "de_DE")
]

eval_combined_dataset = ConcatDataset(datasets)
eval_dataloader = DataLoader(eval_combined_dataset, batch_size=4, shuffle=True)

In [9]:
##### Run this cell

# from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    push_to_hub=False,
    report_to='none',
    logging_steps=10,
    dataloader_num_workers=2,
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [10]:
##### Run this cell

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_combined_dataset,
    eval_dataset=eval_combined_dataset,
    tokenizer=tokenizer,
)

In [11]:
##### Run this cell

trainer.train()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Epoch,Training Loss,Validation Loss
1,1.241800,1.381492
2,1.042400,1.381439
3,0.982300,1.419617


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#sav

TrainOutput(global_step=6750, training_loss=1.1985059789728236, metrics={'train_runtime': 5304.2999, 'train_samples_per_second': 5.09, 'train_steps_per_second': 1.273, 'total_flos': 7314062966784000.0, 'train_loss': 1.1985059789728236, 'epoch': 3.0})

In [12]:
##### Run this cell

from google.colab import drive
drive.mount('/content/drive')
torch.save(model.state_dict(), '/content/drive/MyDrive/Model_weights_MBart.pth')

Mounted at /content/drive


In [13]:
##### Run this cell

test_outputs_df = pd.read_csv('/content/Test output_evaluation.csv')
test_outputs_df['MBart-fine_tuned version'] = None
test_outputs_df

,Unnamed: 0,source,target,MBart-basic_pre-trained version,MBart-fine_tuned version
0,0,But we feel that it would be wiser not to set ...,"<es_XX> Sin embargo, a nuestro juicio, es prud...","['Sin embargo, consideramos que sería más sens...",None
1,1,Subject: 'Mainstreaming' in EU aid policy In 1...,<de_DE> Betrifft: Einbeziehung der Gleichstell...,"['Betrifft: ""Mainstreaming"" in die EU-Hilfepol...",None
2,2,I would like to emphasise the strong political...,<de_DE> Ich möchte besonders auf die vom Europ...,['Ich möchte die starke politische Botschaft d...,None
3,3,It would be very difficult for the Community t...,<de_DE> Es wäre angesichts der unterschiedlich...,"['Es wäre für die Gemeinschaft sehr schwierig,...",None
4,4,We are delighted today to see the progress thi...,<nl_XX> Het doet ons genoegen dat deze gedacht...,['We zijn vandaag blij de vooruitgang te zien ...,None
...,...,...,...,...,...
4417,4417,"In the stance that we took at Seattle, why suc...",<nl_XX> Waarom werd er in het door ons in Seat...,"[""In de houding die we in Seattle hebben genom...",None
4418,4418,The vote will take place tomorrow at 11.30 a.m.,<nl_XX> De stemming vindt morgen om 11.30 uur ...,['De stemming zal morgen om 11.30 uur plaatsvi...,None
4419,4419,On Friday of this week we have one single item...,<es_XX> En el viernes de esta semana tenemos u...,['El viernes de esta semana tenemos un único p...,None
4420,4420,The European Union voted against that motion.,<de_DE> Die Europäische Union stimmte gegen di...,['Die Europäische Union hat gegen diesen Vorsc...,None


In [15]:
##### Run this cell

model = MBartForConditionalGeneration.from_pretrained('facebook/mbart-large-50-one-to-many-mmt')
tokenizer = MBartTokenizer.from_pretrained('facebook/mbart-large-50-one-to-many-mmt')
model.load_state_dict(torch.load('/content/drive/MyDrive/Model_weights_MBart.pth',map_location=torch.device('cuda')), strict=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'MBart50Tokenizer'. 
The class this function is called from is 'MBartTokenizer'.


<All keys matched successfully>

In [21]:
##### Run this cell
model.to('cuda')
for i in tqdm(range(len(test_outputs_df)), desc="Translating Test data"):
    encoded_en_text = tokenizer(test_outputs_df['source'].iloc[i], return_tensors="pt",truncation = True).to('cuda')
    generated_tokens = model.generate(**encoded_en_text, forced_bos_token_id=tokenizer.lang_code_to_id[test_outputs_df['target'].iloc[i][0:7][1:-1]]).to('cuda')
    test_outputs_df['MBart-fine_tuned version'].iloc[i] = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

Translating Test data:   0%|          | 0/4422 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
<ipython-input-21-27d435e1e231>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_outputs_df['MBart-fine_tuned version'].iloc[i] = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
<ipython-input-21-27d435e1e231>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_outputs_df['MBart-fine_tuned version'].iloc[i] = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
<ipython-input-21-27d435e1e231>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:

In [22]:
##### Run this cell

test_outputs_df.to_csv('/content/drive/MyDrive/MBART_outputs.csv')

In [28]:
test_outputs_df['MBart-fine_tuned version'] = test_outputs_df['MBart-fine_tuned version'].apply(lambda x: x[0] if isinstance(x, list) else x)

In [29]:
##### Run this cell

bleu_metric = evaluate.load("bleu")
rouge_metric = evaluate.load("rouge")
meteor_metric = evaluate.load("meteor")

predictions = test_outputs_df['MBart-fine_tuned version'].to_list()
references = test_outputs_df['target'].apply(lambda x: x[7:]).to_list()


bleu_score = bleu_metric.compute(predictions=predictions, references=references)
print(f"BLEU score: {bleu_score}")

# Compute ROUGE score
rouge_score = rouge_metric.compute(predictions=predictions, references=references)
print(f"ROUGE score: {rouge_score}")

# Compute METEOR score
meteor_score = meteor_metric.compute(predictions=predictions, references=references)
print(f"METEOR score: {meteor_score}")


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


BLEU score: {'bleu': 0.286440362065415, 'precisions': [0.5823854273013015, 0.3348114190456345, 0.2229183613423562, 0.15509888251066203], 'brevity_penalty': 0.999638320363694, 'length_ratio': 0.9996383857540059, 'translation_length': 124397, 'reference_length': 124442}
ROUGE score: {'rouge1': 0.5675624974078326, 'rouge2': 0.3454443940774692, 'rougeL': 0.517333973985736, 'rougeLsum': 0.5174150280247815}
METEOR score: {'meteor': 0.5273130348687682}


In [ ]:
##########################################################################################

### Basic SEQ-2-SEQ using BPE

In [ ]:
#### Run this cell

# Path to BPE model file (.model) and (.bin)

# Loading Multilingual BPE pre-trained model
# bpemb_multi = BPEmb(model_file='/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/multi.wiki.bpe.vs100000.model',emb_file='/Users/marianivethaantonypushparaj/Desktop/Pattern Recognition/multi.wiki.bpe.vs100000.d300.w2v.bin',dim=300)
bpemb_multi = BPEmb(model_file='multi.wiki.bpe.vs100000.model',emb_file='multi.wiki.bpe.vs100000.d300.w2v.bin',dim=300)



In [ ]:
# Example testing of model - English
text = "Hello, world! This is a test."
tokens = bpemb_multi.preprocess(text)
tokens = bpemb_multi.encode(text)
print(tokens)
print(bpemb_multi.decode(tokens))
# print('ID encoded:',pairs)
# decode_pairs = bpemb_multi.decode_ids()
# bpemb_multi.decode(decode_pairs)



['▁h', 'ello', ',', '▁world', '!', '▁this', '▁is', '▁a', '▁test', '.']
hello, world! this is a test.


AttributeError: 'list' object has no attribute 'decode_ids'

In [ ]:
#### Run this cell


train_data = pd.read_csv('Train_data.csv')
val_data = pd.read_csv('Validation_data.csv')
test_data = pd.read_csv('Test_data.csv')

train_data = train_data[['source','target']]
val_data = val_data[['source','target']]
test_data = test_data[['source','target']]

In [ ]:
#### Run this cell

for i in range(len(train_data['target'])):
    if train_data['target'].iloc[i][0:7] == '<nl_XX>':
        train_data['source'].iloc[i] = 'Translate to Dutch: ' + train_data['source'].iloc[i]
        train_data['target'].iloc[i] = 'Dutch:' + train_data['target'].iloc[i][7:]
    if train_data['target'].iloc[i][0:7] == '<de_DE>':
        train_data['source'].iloc[i] = 'Translate to German: ' + train_data['source'].iloc[i]
        train_data['target'].iloc[i] = 'German:' + train_data['target'].iloc[i][7:]
    if train_data['target'].iloc[i][0:7] == '<es_XX>':
        train_data['source'].iloc[i] = 'Translate to Spanish: ' + train_data['source'].iloc[i]
        train_data['target'].iloc[i] = 'Spanish:' + train_data['target'].iloc[i][7:]

In [ ]:
#### Run this cell


for i in range(len(val_data['target'])):
    if val_data['target'].iloc[i][0:7] == '<nl_XX>':
        val_data['source'].iloc[i] = 'Translate to Dutch: ' + val_data['source'].iloc[i]
        val_data['target'].iloc[i] = 'Dutch:' + val_data['target'].iloc[i][7:]
    if val_data['target'].iloc[i][0:7] == '<de_DE>':
        val_data['source'].iloc[i] = 'Translate to German: ' + val_data['source'].iloc[i]
        val_data['target'].iloc[i] = 'German:' + val_data['target'].iloc[i][7:]
    if val_data['target'].iloc[i][0:7] == '<es_XX>':
        val_data['source'].iloc[i] = 'Translate to Spanish: ' + val_data['source'].iloc[i]
        val_data['target'].iloc[i] = 'Spanish:' + val_data['target'].iloc[i][7:]

In [ ]:
#### Run this cell

for i in range(len(test_data['target'])):
    if test_data['target'].iloc[i][0:7] == '<nl_XX>':
        val_data['source'].iloc[i] = 'Translate to Dutch: ' + test_data['source'].iloc[i]
        test_data['target'].iloc[i] = test_data['target'].iloc[i][7:]
    if test_data['target'].iloc[i][0:7] == '<de_DE>':
        test_data['source'].iloc[i] = 'Translate to German: ' + test_data['source'].iloc[i]
        test_data['target'].iloc[i] = test_data['target'].iloc[i][7:]
    if test_data['target'].iloc[i][0:7] == '<es_XX>':
        test_data['source'].iloc[i] = 'Translate to Spanish: ' + test_data['source'].iloc[i]
        test_data['target'].iloc[i] = test_data['target'].iloc[i][7:]

In [ ]:
#### Run this cell

train_data = train_data.sample(frac=1).reset_index(drop=True)
val_data = val_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

In [ ]:
#### Run this cell

train_data = train_data[:10000]
val_data = val_data[:1200]
test_data = test_data[:1000]

In [ ]:
#### Run this cell

SOS_token = bpemb_multi.BOS
EOS_token = bpemb_multi.EOS

# device = torch.device('mps') if torch.backends.mps.is_available() else torch.device('cpu')
# device = 'cuda' if torch.cuda.is_available() else 'cpu'

def pre_process_bpemb(text):
    sentence = bpemb_multi.preprocess(text)
    encoded_text = bpemb_multi.encode_ids(sentence)
    encoded_text.append(EOS_token)
    return torch.tensor(encoded_text, dtype=torch.long, device=device)

def decoding_sentence(text):
    sentence = bpemb_multi.decode_ids(text)
    return sentence

In [ ]:
device

device(type='xla', index=0)

In [ ]:
#### Run this cell

def pad_tensor_to_length(tensor, length=300):
    # Calculate the padding size
    pad_size = length - tensor.size(0)

    if pad_size > 0:
        # Pad tensor with zeros on the right side
        padded_tensor = F.pad(tensor, (0, pad_size), "constant", 0)
    else:
        # If tensor is already longer than the required length, truncate it
        padded_tensor = tensor[:length]

    return padded_tensor

In [ ]:
#### Run this cell

encoded_src_text = []
encoded_tgt_text = []

for i in range(len(train_data)):
    encoded_src_text.append(pad_tensor_to_length(pre_process_bpemb(train_data['source'].iloc[i])))
    encoded_tgt_text.append(pad_tensor_to_length(pre_process_bpemb(train_data['target'].iloc[i])))

print('Train dim:',len(encoded_src_text))
print('Train dim:',len(encoded_tgt_text))

# encoded_src_text = pad_sequence(encoded_src_text, batch_first=True, padding_value=0)
# encoded_tgt_text = pad_sequence(encoded_tgt_text, batch_first=True, padding_value=0)

train_sentences = list(zip(encoded_src_text,encoded_tgt_text))
train_dataloader = DataLoader(train_sentences, batch_size=64, shuffle=True)

encoded_src_text = []
encoded_tgt_text = []

for i in range(len(val_data)):
        encoded_src_text.append(pad_tensor_to_length(pre_process_bpemb(val_data['source'].iloc[i])))
        encoded_tgt_text.append(pad_tensor_to_length(pre_process_bpemb(val_data['target'].iloc[i])))


print('Val dim:',len(encoded_src_text))
print('Val dim:',len(encoded_tgt_text))

val_sentences = list(zip(encoded_src_text,encoded_tgt_text))
val_dataloader = DataLoader(val_sentences, batch_size=64, shuffle=True)

Train dim: 10000
Train dim: 10000
Val dim: 1200
Val dim: 1200


In [ ]:
#### Run this cell

class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, dropout_p=0.1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True, dropout=dropout_p, bidirectional=False)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, input_seq):
        embedded = self.dropout(self.embedding(input_seq))
        output, hidden = self.gru(embedded)
        return output, hidden

In [ ]:
#### Run this cell

class BahdanauAttention(nn.Module):
    def __init__(self, hidden_size):
        super(BahdanauAttention, self).__init__()
        self.Wa = nn.Linear(hidden_size, hidden_size)
        self.Ua = nn.Linear(hidden_size, hidden_size)
        self.Va = nn.Linear(hidden_size, 1)

    def forward(self, query, keys):
        scores = self.Va(torch.tanh(self.Wa(query) + self.Ua(keys)))
        scores = scores.squeeze(2).unsqueeze(1)

        weights = F.softmax(scores, dim=-1)
        context = torch.bmm(weights, keys)

        return context, weights

In [ ]:
#### Run this cell

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1):
        super(AttnDecoderRNN, self).__init__()
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.attention = BahdanauAttention(hidden_size)
        self.gru = nn.GRU(2 * hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout_p)

    def forward(self, encoder_outputs, encoder_hidden, target_tensor):
        batch_size = encoder_outputs.size(0)
        decoder_input = torch.empty(batch_size, 1, dtype=torch.long, device=device).fill_(SOS_token)
        decoder_hidden = encoder_hidden
        decoder_outputs = []
        attentions = []

        for i in range(300):
            decoder_output, decoder_hidden, attn_weights = self.forward_step(
                decoder_input, decoder_hidden, encoder_outputs
            )
            decoder_outputs.append(decoder_output)
            attentions.append(attn_weights)

            if target_tensor is not None:
                # Teacher forcing: Feed the target as the next input
                decoder_input = target_tensor[:, i].unsqueeze(1) # Teacher forcing
            else:
                # Without teacher forcing: use its own predictions as the next input
                _, topi = decoder_output.topk(1)
                decoder_input = topi.squeeze(-1).detach()  # detach from history as input

        decoder_outputs = torch.cat(decoder_outputs, dim=1)
        decoder_outputs = F.log_softmax(decoder_outputs, dim=-1)
        attentions = torch.cat(attentions, dim=1)

        return decoder_outputs, decoder_hidden, attentions


    def forward_step(self, input, hidden, encoder_outputs):
        embedded =  self.dropout(self.embedding(input))

        query = hidden.permute(1, 0, 2)
        context, attn_weights = self.attention(query, encoder_outputs)
        input_gru = torch.cat((embedded, context), dim=2)

        output, hidden = self.gru(input_gru, hidden)
        output = self.out(output)

        return output, hidden, attn_weights

In [ ]:
#### Run this cell

SOS_token = bpemb_multi.BOS
EOS_token = bpemb_multi.EOS

In [ ]:
#### Run this cell

def train_epoch(dataloader, encoder, decoder, encoder_optimizer,
          decoder_optimizer, criterion):

    total_loss = 0
    for data in dataloader:

        input_tensor, target_tensor = data

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        encoder_outputs, encoder_hidden = encoder(input_tensor)
        decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor)

        loss = criterion(
            decoder_outputs.view(-1, decoder_outputs.size(-1)),
            target_tensor.view(-1)
        )
        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
#### Run this cell

import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [ ]:
#### Run this cell

def evaluate(dataloader, encoder, decoder, criterion):
    total_loss = 0
    with torch.no_grad():
        for data in dataloader:
            input_tensor, target_tensor = data

            encoder_outputs, encoder_hidden = encoder(input_tensor)
            decoder_outputs, _, _ = decoder(encoder_outputs, encoder_hidden, target_tensor=None)

            loss = criterion(
                decoder_outputs.view(-1, decoder_outputs.size(-1)),
                target_tensor.view(-1)
            )

            total_loss += loss.item()

    return total_loss / len(dataloader)


In [ ]:
#### Run this cell

def train(train_dataloader, valid_dataloader, encoder, decoder, n_epochs, learning_rate=0.001, print_every=1, plot_every=1):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    valid_losses = []

    encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
    criterion = nn.NLLLoss()

    epoch_bar = trange(n_epochs, desc="Training and Validating Epochs", unit="epoch")
    i = 0

    for epoch in epoch_bar:
        i += 1
        train_loss = train_epoch(train_dataloader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += train_loss
        plot_loss_total += train_loss

        valid_loss = evaluate(valid_dataloader, encoder, decoder, criterion)
        valid_losses.append(valid_loss)

        if i % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) Train Loss: %.4f' % (timeSince(start, i / n_epochs), i, i / n_epochs * 100, print_loss_avg))

        if i % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

        # Log validation loss
        print('%s (%d %d%%) Valid Loss: %.4f' % (timeSince(start, i / n_epochs), i, i / n_epochs * 100, valid_loss))

        save_checkpoint(i, encoder, decoder, encoder_optimizer, decoder_optimizer)

    showPlot(plot_losses)
    showPlot(valid_losses)

In [ ]:
#### Run this cell

import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:
#### Run this cell

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#### Run this cell

save_dir = '/content/drive/MyDrive/Seq2seq/'
def save_checkpoint(epoch, encoder, decoder, encoder_optimizer, decoder_optimizer):
    encoder_path = f'{save_dir}/encoder_epoch_{epoch}.pth'
    decoder_path = f'{save_dir}/decoder_epoch_{epoch}.pth'
    encoder_optimizer_path = f'{save_dir}/encoder_optimizer_epoch_{epoch}.pth'
    decoder_optimizer_path = f'{save_dir}/decoder_optimizer_epoch_{epoch}.pth'

    torch.save(encoder.state_dict(), encoder_path)
    torch.save(decoder.state_dict(), decoder_path)
    torch.save(encoder_optimizer.state_dict(), encoder_optimizer_path)
    torch.save(decoder_optimizer.state_dict(), decoder_optimizer_path)

In [ ]:
#### Run this cell

hidden_size = 128
batch_size = 64

encoder = EncoderRNN(100000, hidden_size).to(device)
decoder = AttnDecoderRNN(hidden_size, 100000).to(device)

train(train_dataloader,val_dataloader, encoder, decoder,3, print_every=1, plot_every=1)

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
Training and Validating Epochs:   0%|          | 0/3 [01:34<?, ?epoch/s]


RuntimeError: Bad StatusOr access: RESOURCE_EXHAUSTED: XLA:TPU compile permanent error. Ran out of memory in memory space hbm. Used 13.26G of 7.48G hbm. Exceeded hbm capacity by 5.78G.

Total hbm usage >= 13.78G:
    reserved        530.00M 
    program          13.26G 
    arguments            0B 

Output size 0B; shares 0B with arguments.

Program hbm requirement 13.26G:
    HLO temp         13.26G (99.9% utilization: Unpadded (12.22G) Padded (12.23G), 7.8% fragmentation (1.03G))

  Largest program allocations in hbm:

  1. Size: 6.11G
     Shape: f32[64,256,100000]{2,0,1:T(8,128)}
     Unpadded size: 6.10G
     Extra memory due to padding: 6.00M (1.0x expansion)
     XLA label: fusion.635.remat_uncompressed = copy(fusion.635.remat_compressed)
     Allocation type: HLO temp
     ==========================

  2. Size: 6.10G
     Shape: f32[64,256,100000]{1,2,0:T(8,128)}
     Unpadded size: 6.10G
     XLA label: fusion.635.remat_compressed = copy(fusion.635)
     Allocation type: HLO t

### Testing Pre-trained models

In [ ]:
from transformers import MarianMTModel, MarianTokenizer

# Load the tokenizer and model for English to Spanish translation
model_name = 'Helsinki-NLP/opus-mt-en-es'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

/Users/marianivethaantonypushparaj/Library/Python/3.9/lib/python/site-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
print(data_spa['Spanish'][100])
print(data_spa['Spanish'][1000])
print(data_spa['Spanish'][1])

Señor Presidente, en mi opinión, este debate conjunto es el testimonio más vivo y valioso de ello.
Podríamos hacer bastante más, teniendo en cuenta que hemos asumido grandes compromisos en lo relativo a la reducción de las emisiones de CO2 y de gases de efecto invernadero, además de una reducción de la dependencia de las importaciones de energía.
Permítanme reseñar brevemente algunos de los fundamentos y principios en los que se sostiene esta directiva.


In [ ]:
# Sample English sentences
english_sentences = data_spa['English']

# Tokenize the input sentences
input_ids = tokenizer(english_sentences, return_tensors="pt", padding=True, truncation=True).input_ids

# Generate output ids
output_ids = model.generate(input_ids, num_beams=4, max_length=50, early_stopping=True)

# Decode the output ids to get the translations
translations = [tokenizer.decode(ids, skip_special_tokens=True) for ids in output_ids]

# Print the translations
for original, translation in zip(english_sentences, translations):
    print(f"Original: {original}")
    print(f"Translation: {translation}")
    print()


Original: In my opinion, Mr President, this joint debate fully bears this out.
Translation: En mi opinión, señor Presidente, este debate conjunto lo confirma plenamente.

Original: We could do rather more, considering that we have made great commitments in reducing CO2 and greenhouse gases and to reduce dependence on energy imports.
Translation: Podríamos hacer algo más, considerando que hemos asumido grandes compromisos para reducir el CO2 y los gases de efecto invernadero y para reducir la dependencia de las importaciones de energía.

Original: Allow me briefly to comment on some of the assumptions and principles behind the present directive.
Translation: Permítanme comentar brevemente algunos de los supuestos y principios que subyacen a la presente Directiva.



# References

1. https://medium.com/@mail4sameera/multilingual-language-models-in-natural-language-processing-nlp-with-python-9a6d1fda4adc
2. https://medium.com/@jessica_lopez/understanding-greedy-search-and-beam-search-98c1e3cd821d
3. https://github.com/ymoslem/OpenNMT-Tutorial/blob/main/1-NMT-Data-Processing.ipynb
4. https://github.com/bheinzerling/bpemb?tab=readme-ov-file
5. https://github.com/google/sentencepiece/tree/master/src
6.
